In [1]:
import torch
import numpy as np

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import ShapeNet

import torch_geometric.transforms as T
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from torch_geometric.utils import scatter

from torch_geometric.nn import GCNConv

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# DATA HANDLING

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

print(data)

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

print(data)

data.validate(raise_on_error=True)

print(data.keys)

print(data['x'])

for key, item in data:
    print(f'{key} found in data')

print(f"n_nodes={data.num_nodes} n_edges={data.num_edges}")

Data(x=[3, 1], edge_index=[2, 4])
Data(x=[3, 1], edge_index=[2, 4])
['x', 'edge_index']
tensor([[-1.],
        [ 0.],
        [ 1.]])
x found in data
edge_index found in data
n_nodes=3 n_edges=4


In [6]:
dataset = Planetoid(root='./data/datasets', name='Cora')
print(len(dataset))
data = dataset[0]
print(data)
dataset = dataset.shuffle()
train_dataset = dataset[:540]
test_dataset = dataset[540:]
loader = DataLoader(dataset, batch_size=32, shuffle=True)

1
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [7]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print(f"epoch_{epoch} loss={loss}")

epoch_0 loss=1.9496701955795288
epoch_1 loss=1.8450946807861328
epoch_2 loss=1.7104085683822632
epoch_3 loss=1.5497912168502808
epoch_4 loss=1.3972631692886353
epoch_5 loss=1.2623569965362549
epoch_6 loss=1.1332038640975952
epoch_7 loss=0.9775575995445251
epoch_8 loss=0.8800915479660034
epoch_9 loss=0.8240165710449219
epoch_10 loss=0.6897128820419312
epoch_11 loss=0.5907596349716187
epoch_12 loss=0.5582388639450073
epoch_13 loss=0.5111879706382751
epoch_14 loss=0.4274301826953888
epoch_15 loss=0.35105642676353455
epoch_16 loss=0.31483960151672363
epoch_17 loss=0.325553297996521
epoch_18 loss=0.2749057710170746
epoch_19 loss=0.22495166957378387
epoch_20 loss=0.22910551726818085
epoch_21 loss=0.18797174096107483
epoch_22 loss=0.16249911487102509
epoch_23 loss=0.15115945041179657
epoch_24 loss=0.13561537861824036
epoch_25 loss=0.13039812445640564
epoch_26 loss=0.1383766084909439
epoch_27 loss=0.10946402698755264
epoch_28 loss=0.12564687430858612
epoch_29 loss=0.12649892270565033
epoch_30 

In [14]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8080
